## 1. Load the Data

Korpora에서 제공하는 NSMCDataset의 테스트 데이터셋에서 20000개의 데이터만 사용하여 학습 진행

In [3]:
import numpy as np
import pandas as pd
from Korpora import Korpora

corpus = Korpora.load("nsmc")
df = pd.DataFrame(corpus.test).sample(20000, random_state=42)

# train : val : test 를 6 : 2 : 2 로 분리
train_df, val_df, test_df = np.split(
    df.sample(frac=1, random_state=42), [int(0.6 * len(df)), int(0.8 * len(df))]
)

# 출력 확인
print(train_df.head(5).to_markdown())
print(f"train: {len(train_df)}, val: {len(val_df)}")
print(f"test: {len(test_df)}")


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at /home/ho/Korpora/nsmc/ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at /home/ho/Korpora/n

## 2. Dataset & DataLoader

Load해온 NSMCDataset을 DataLoader를 통해 배치 단위로 불러올 수 있도록 하는 함수.

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

def make_dataset(data, tokenizer, device):
    tokenized = tokenizer(
        text = data.text.tolist(),
        padding="longest", # 배치 내에서 가장 긴 sequence 길이로 통일
        truncation=True, # 모델이 처리할 수 있는 최대 길이로 입력 조정 (BERT의 경우, 512)
        return_tensors="pt"
    )

    input_ids = tokenized["input_ids"].to(device)
    att_masks = tokenized["attention_mask"].to(device)
    labels = torch.tensor(data.label.values, dtype=torch.long).to(device)
    
    return TensorDataset(input_ids, att_masks, labels)

def get_dataloader(dataset, sampler, batch_size):
    data_sampler = sampler(dataset)
    data_loader = DataLoader(dataset, sampler=data_sampler, batch_size=batch_size)

    return data_loader

## 3. Check the Netwrok Architecture

이후 실험할 BERT, GPT2에 대한 Network 구조를 확인하기 위한 함수

In [ ]:
def check_network(net):
    name, _ = next(net.named_children())
    for main_name, main_module in net.named_children():
        print(main_name)

        for sub_name, sub_module in main_module.named_children():
            print("- ", sub_name)

            for ssub_name, ssub_module in sub_module.named_children():
                print("|    - ", ssub_name)

                for sssub_name, _ in ssub_module.named_children():
                    print("|    |   - ", sssub_name)
    
    return name

## 4. Train & Test Function

모델 훈련 및 테스트 코드

In [ ]:
import numpy as np
import torch.nn as nn
from tqdm.notebook import tqdm

def calc_accuracy(preds, labels):
    pred_flatten = np.argmax(preds, axis=1).flatten()
    labels_flatten = labels.flatten()

    return np.sum(pred_flatten == labels_flatten) / len(labels_flatten)

def train(net, optimizer, data_loader):
    net.train()
    train_loss = 0.0

    for input_ids, att_mask, labels in tqdm(data_loader):
        optimizer.zero_grad()

        outputs = net(
            input_ids=input_ids,
            attention_mask=att_mask,
            labels=labels
        )

        loss = outputs.loss
        train_loss += loss.item()
        
        loss.backward()
        optimizer.step()
    
    train_loss /= len(data_loader)
    return train_loss

def eval(net, data_loader):
    net.eval()
    criterion = nn.CrossEntropyLoss()
    val_loss, val_acc = 0.0, 0.0

    with torch.no_grad():
        for input_ids, att_mask, labels in tqdm(data_loader):
            outputs = net(
                input_ids=input_ids,
                attention_mask=att_mask,
                labels=labels
            )
            
            logits = outputs.logits
            loss = criterion(logits, labels)
            
            logits = logits.detach().cpu().numpy()
            label_ids = labels.to("cpu").numpy()
            acc = calc_accuracy(logits, label_ids)

            val_loss += loss
            val_acc += acc

    val_loss /= len(data_loader)
    val_acc /= len(data_loader)

    return val_loss, val_acc

In [ ]:
def test(net, main_name, test_loader):    
    net.load_state_dict(torch.load(f"./pth/{main_name}.pth"))

    test_loss, test_acc = eval(net, test_loader)
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


## 5. Bert

Bert를 통해 Self-Supervised Learning으로 Transfer Learning 된 모델과 그렇지 않은 모델 성능 비교

### 5-1. Hyper-Parameter 설정

In [ ]:
EPOCHS = 5
BATCH_SIZE = 32
LR = 1e-5
EPS = 1e-8

device = "cuda" if torch.cuda.is_available() else "cpu"
pretrained = "bert-base-multilingual-cased"

### 5-2. DataLoader 불러오기

이때, tokenizer는 pretrained된 값을 사용 (오로지, 모델의 pretrained 여부에 따른 성능 차이 확인을 위해서)

In [ ]:
from transformers import AutoTokenizer
from torch.utils.data import RandomSampler, SequentialSampler

def prepare_data(pretrained):
    tokenizer = AutoTokenizer.from_pretrained(
        pretrained_model_name_or_path=pretrained,
        do_lower_case=False
    )

    if pretrained == "gpt2":
        tokenizer.pad_token = tokenizer.eos_token

    train_dataset = make_dataset(train_df, tokenizer, device)
    train_loader = get_dataloader(train_dataset, RandomSampler, BATCH_SIZE)

    val_dataset = make_dataset(val_df, tokenizer, device)
    val_loader = get_dataloader(val_dataset, SequentialSampler, BATCH_SIZE)

    test_dataset = make_dataset(test_df, tokenizer, device)
    test_loader = get_dataloader(test_dataset, SequentialSampler, BATCH_SIZE)

    return train_loader, val_loader, test_loader

train_loader, val_loader, test_loader = prepare_data(pretrained)
for input_ids, attn_masks, labels in train_loader:
    print(f"{input_ids.shape}, {attn_masks.shape}, {labels.shape}")
    break

### 5-3. Supervised Learning

In [ ]:
# 모델 정의
from transformers import BertConfig, BertForSequenceClassification

config = BertConfig(
    vocab_size= 119547,
    num_labels=2
)

net = BertForSequenceClassification(config=config).to(device)
main_name = check_network(net)

In [ ]:
# optimizer 정의
from torch.optim import AdamW
optimizer = AdamW(net.parameters(), lr=LR, eps=EPS)

In [ ]:
# 훈련 코드

best_loss = 1e9
for epoch in range(EPOCHS):
    train_loss = train(net, optimizer, train_loader)
    val_loss, val_acc = eval(net, val_loader)

    print(f"Epoch [{epoch+1}/{EPOCHS}]")
    print(f"  Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}")
    
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(net.state_dict(), f"./pth/{main_name}.pth")

In [ ]:
# 테스트
net = BertForSequenceClassification(config=config).to(device)
test(net, main_name, test_loader)

### 5-4. Self-Supervised Learning

In [4]:
df = train_df + val_df

In [5]:
# NSP를 위해 문장 Pair 생성
import random

def create_sent_pairs(df):
    sent_a = []
    sent_b = []
    labels = []

    for i in range(len(df) - 1):
        sent_a.append(df.iloc[i]['text'])
        sent_b.append(df.iloc[i + 1]['text'])
        labels.append(1)  # 연속 문장
        sent_a.append(df.iloc[i]['text'])
        sent_b.append(df.iloc[random.randint(0, len(df)-1)]['text'])
        labels.append(0)  # 무작위 문장

    return sent_a, sent_b, labels

sent_a, sent_b, labels = create_sent_pairs(df)

In [ ]:
# MLP와 NSP를 적용하기 위한 데이터셋
from torch.utils.data import Dataset, DataLoader

class NSPandMLMDataset(Dataset):
    def __init__(self, tokenizer, text_a, text_b, labels):
        self.encodings = tokenizer(text_a, text_b, truncation=True, padding=True, max_length=512)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['next_sentence_label'] = torch.LongTensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# 모델 정의
from transformers import AutoModelForSequenceClassification

net = AutoModelForSequenceClassification.from_pretrained(
        pretrained, num_labels=2
    ).to(device)
main_name = check_network(net)
main_name += "_pretrained"

In [ ]:
# optimizer 정의
from torch.optim import AdamW
optimizer = AdamW(net.parameters(), lr=LR, eps=EPS)

In [ ]:
# 훈련
main(net, main_name, optimizer, train_loader, val_loader)

In [ ]:
# 테스트
net = AutoModelForSequenceClassification.from_pretrained(
        pretrained, num_labels=2
    ).to(device)
test(net, main_name, test_loader)